In [1]:
from trainers.ngboost_trainer import NGBoostTrainer
from estimators.ngboost_estimator import NGBoostEstimator

from trainers.sklearn_random_forest import SklearnRandomForestTrainer


import pandas as pd
import nrel.routee.powertrain as pt
# import core.powertrain_type as pt1
from core.model_config import ModelConfig
import math




In [ ]:
def test_ngboost_rate(self):
        trainer = NGBoostTrainer()

        vehicle_model = trainer.train(self.df, self.rate_config)

        r1 = vehicle_model.predict(self.df)
        energy1 = round(r1.gallons_fastsim.sum(), 2)

        # test out writing and reading to file
        outfile = self.out_path / "model_rate.json"
        vehicle_model.to_file(outfile)
        new_vehicle_model = pt.load_model(outfile)
        outfile.unlink()

        # test writing inner estimator to file
        outfile = self.out_path / "estimator_rate.json"
        estimator = list(vehicle_model.estimators.values())[0]
        estimator.to_file(outfile)
        _ = NGBoostEstimator.from_file(outfile)
        outfile.unlink()

        r2 = new_vehicle_model.predict(self.df)
        energy2 = round(r2.gallons_fastsim.sum(), 2)

        self.assertTrue(math.isclose(energy1, energy2))

    

In [ ]:
def test_ngboost_raw(self):
    trainer = NGBoostTrainer()

    vehicle_model = trainer.train(self.df, self.raw_config)

    r1 = vehicle_model.predict(self.df)
    energy1 = round(r1.gallons_fastsim.sum(), 2)

    # test out writing and reading to file
    outfile = self.out_path / "model_raw.json"
    vehicle_model.to_file(outfile)
    new_vehicle_model = pt.load_model(outfile)
    outfile.unlink()

    # test writing inner estimator to file
    outfile = self.out_path / "estimator_raw.json"
    estimator = list(vehicle_model.estimators.values())[0]
    estimator.to_file(outfile)
    _ = NGBoostEstimator.from_file(outfile)
    outfile.unlink()

    r2 = new_vehicle_model.predict(self.df)
    energy2 = round(r2.gallons_fastsim.sum(), 2)

    self.assertTrue(math.isclose(energy1, energy2))

In [2]:
# import core.features as pt


In [3]:
ngb = NGBoostTrainer()

In [4]:
df = pd.read_csv('/Users/speddire/Documents/routeE_project/routee-powertrain/tests/routee-powertrain-test-data/sample_train_data.csv')
df
df['gallons_fastsim_rate'] = df['gallons_fastsim']/df['miles']
df
# NGBoostTrainer.inner_train(X_train,y_train)

,speed_mph,grade_dec,miles,gallons_fastsim,trip_id,road_class,gallons_fastsim_rate
0,7.632068,-0.008963,0.015469,0.000813,1,3,0.052579
1,6.329613,-0.047001,0.003516,0.000149,1,3,0.042404
2,12.248512,0.000000,0.003402,0.000074,1,4,0.021898
3,23.752604,-0.000463,0.019768,0.002194,1,1,0.111003
4,46.024926,-0.004641,0.038378,0.000970,1,0,0.025276
...,...,...,...,...,...,...,...
120,7.415272,-0.001339,0.006128,0.000224,1,4,0.036493
121,27.685268,-0.030748,0.023170,0.000224,1,0,0.009653
122,51.322545,-0.008490,0.028510,0.000357,1,4,0.012521
123,50.431920,-0.010533,0.028015,0.000610,1,1,0.021758


In [5]:
feature_set_1 = pt.FeatureSet([pt.DataColumn(name="speed_mph", units="mph")])
feature_set_2 = pt.FeatureSet([
    pt.DataColumn(name="speed_mph", units="mph"),
    pt.DataColumn(name="grade_dec", units="decimal")
])
feature_set_3 = [
    pt.DataColumn(name="speed_mph", units="mph"),
    pt.DataColumn(name="grade_dec", units="decimal"),
    pt.DataColumn(name="road_class", units="category")
]
features = [
    feature_set_1,
    # feature_set_2,
    # feature_set_3
]

In [6]:
distance = pt.DataColumn(name="miles", units="miles")

In [7]:
# energy_target = pt.DataColumn(
#     name="gallons_fastsim", 
#     units="gallons_gasoline", 
# )

# energy_target = pt.TargetSet(targets=[pt.DataColumn(
#     name="gallons_fastsim", 
#     units="gallons_gasoline", 
# )])

energy_target = pt.TargetSet(targets=[pt.DataColumn(
    name="gallons_fastsim_rate", 
    units="gallons_gasoline", 
)])

In [8]:
predict_method = "rate"

In [9]:
config = ModelConfig(
    vehicle_description="Test Vehicle",
    powertrain_type=pt.PowertrainType.ICE,
    feature_sets=features,
    distance=distance,
    target=energy_target,
    test_size=0.2,
    predict_method=predict_method
)

In [10]:
trainer = NGBoostTrainer()
# trainer = SklearnRandomForestTrainer()

In [11]:
test_vehicle = trainer.train(df, config)

[iter 0] loss=-1.8281 val_loss=0.0000 scale=2.0000 norm=0.9229
[iter 20] loss=-2.0561 val_loss=0.0000 scale=2.0000 norm=0.6237
[iter 40] loss=-2.1728 val_loss=0.0000 scale=2.0000 norm=0.6440
[iter 60] loss=-2.2729 val_loss=0.0000 scale=2.0000 norm=0.6562
[iter 80] loss=-2.3627 val_loss=0.0000 scale=2.0000 norm=0.6721


In [11]:
test_vehicle = trainer.train(df, config)

[iter 0] loss=-4.9082 val_loss=0.0000 scale=1.0000 norm=0.6360
[iter 20] loss=-5.1415 val_loss=0.0000 scale=2.0000 norm=0.9461
[iter 40] loss=-5.2719 val_loss=0.0000 scale=1.0000 norm=0.4275
[iter 60] loss=-5.3703 val_loss=0.0000 scale=2.0000 norm=0.8176
[iter 80] loss=-5.4834 val_loss=0.0000 scale=2.0000 norm=0.7833
[0.0005261  0.00246761 0.00143965 0.00166262 0.00143965 0.00045715
 0.00055562 0.00195282 0.00199612 0.00071231 0.0014466  0.00199612
 0.0005261  0.00054289 0.00107358 0.00106369 0.00199612 0.0005261
 0.00071231 0.00139962 0.00055562 0.00055562 0.00107358 0.00143965
 0.0005261  0.00246761]
1      0.003516
7      0.045503
11     0.037550
12     0.014632
33     0.148708
34     0.011433
35     0.023094
43     0.028105
50     0.058882
52     0.003330
53     0.010450
55     0.045125
62     0.006424
67     0.023140
69     0.030986
73     0.005803
80     0.019294
88     0.004237
104    0.005474
112    0.121534
113    0.036744
115    0.014618
116    0.010325
118    0.038014
120   

In [16]:
test_vehicle.predict(df)

,gallons_fastsim_rate,gallons_fastsim_rate_std
0,0.000737,0.000272
1,0.000164,0.000062
2,0.000130,0.000084
3,0.001134,0.000775
4,0.001045,0.000741
...,...,...
120,0.000292,0.000108
121,0.001201,0.000876
122,0.000754,0.000576
123,0.000741,0.000566


In [15]:
dir(test_vehicle)
# test_vehicle

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'contour',
 'errors',
 'estimators',
 'feature_set_lists',
 'feature_sets',
 'from_dict',
 'from_file',
 'from_url',
 'metadata',
 'predict',
 'to_dict',
 'to_file',
 'visualize_features']

In [14]:
test_vehicle.visualize_features

<bound method Model.visualize_features of ========================================
Model Summary
--------------------
Vehicle description: Test Vehicle
Powertrain type: ICE
Number of estimators: 1
Estimator Summary
--------------------
Feature: speed_mph (mph)
Distance: miles (miles)
Target: gallons_fastsim_rate (gallons_gasoline)
Raw Predicted Consumption: 26.686 (miles/gallons_gasoline)
Real World Predicted Consumption: 22.886 (miles/gallons_gasoline)
Predict Method: RATE
========================================>

In [16]:
test_vehicle.estimators['speed_mph'].__class__.__name__

'NGBoostEstimator'

In [16]:
# test_vehicle.to_file('test.json')

In [17]:
# {'ngboost_model' :test_vehicle.to_dict()}
# {'ngboost_model': test_vehicle.to_dict()}
dict1 = test_vehicle.to_dict()
# test_vehicle.print_som()

In [27]:
dict1.keys()


dict_keys(['metadata', 'errors', 'all_estimators', 'estimator_constructor_type'])

In [31]:
saved_dict = dict1['all_estimators']['speed_mph']['estimator_constructor_type']
saved_dict

'NGBoostEstimator'

In [22]:
# test_vehicle_2 = pt.Model.from_dict(dict1)
# # NGBoostEstimator.from_file(file_path)
# test_vehicle_2
dict1['estimator_constructor_type']

'dict'

In [38]:
type(test_vehicle_2)

estimators.ngboost_estimator.NGBoostEstimator

In [24]:
result = test_vehicle.predict()
(result)

TypeError: Model.predict() missing 1 required positional argument: 'links_df'

In [36]:
test_vehicle.estimators

{'speed_mph': <estimators.ngboost_estimator.NGBoostEstimator at 0x13ea99b90>}

In [19]:
boosty = NGBoostEstimator(test_vehicle)
boosty

In [20]:
print(dir(boosty))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'from_dict', 'from_file', 'model', 'predict', 'to_dict', 'to_file']


In [21]:
boosty.predict(links_df = df, feature_set = feature_set_3, distance = distance, target_set = energy_target)

AttributeError: 'list' object has no attribute 'feature_name_list'

In [22]:
trainer.verbose


True

In [12]:
dataset = ddf[['speed_kph','grade_percent','distance_km','energy_rate_per_100km']].to_numpy()
# dataset

NameError: name 'ddf' is not defined

In [13]:
# X = dataset[:,:-1]
# y = dataset[:,-1]
# y = y.reshape(y.shape[0],1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=609)

X_train = ddf[['speed_kph','grade_percent','distance_km','pseudo_acc']]
y_train = ddf[['energy_rate_per_100km']]

NameError: name 'ddf' is not defined

In [14]:
ddf

NameError: name 'ddf' is not defined

In [15]:
feature1 = pt.DataColumn(name="speed_kph", units="km/h")
feature2 = pt.DataColumn(name="grade_percent", units="%")
feature3 = pt.DataColumn(name="pseudo_acc", units="")
feature4 = pt.DataColumn(name="distance_km", units="km")

# Create a FeatureSet
feature_set = pt.FeatureSet(features=[feature1, feature2, feature3])

config = pt.ModelConfig(
    vehicle_description="ICE car",
    powertrain_type=pt.PowertrainType.ICE,
    feature_sets=feature_set,
    distance=feature4,
    target=pt.TargetSet(targets=[pt.DataColumn(name="energy_rate_per_100km", units="kWh/km")]),
)
# trainer = NGBoostTrainer()
# trainer.inner_train(features=X_train,target=y_train,config=config)

AttributeError: module 'core.features' has no attribute 'ModelConfig'

In [16]:
ngb.train()

TypeError: Trainer.train() missing 2 required positional arguments: 'data' and 'config'

In [1]:
import joblib
import base64
import io

def to_dict(self) -> dict:
    # Serialize the NGBoost model to a byte stream
    byte_stream = io.BytesIO()
    joblib.dump(self, byte_stream)
    byte_stream.seek(0)
    
    # Encode the byte stream to a base64 string
    model_base64 = base64.b64encode(byte_stream.read()).decode('utf-8')
    
    # Create the dictionary with the base64 string
    out_dict = {
        "ngboost_model": model_base64
    }
    return out_dict


In [2]:
import joblib
import base64
import io

@classmethod
def from_dict(cls, in_dict: dict) -> 'NGBoost':
    # Get the base64 encoded model string from the dictionary
    model_base64 = in_dict.get("ngboost_model")
    if model_base64 is None:
        raise ValueError("Model file must contain ngboost model at key: 'ngboost_model'")
    
    # Decode the base64 string to a byte stream
    byte_stream = io.BytesIO(base64.b64decode(model_base64))
    
    # Deserialize the byte stream to an NGBoost model
    ngboost_model = joblib.load(byte_stream)
    
    return ngboost_model


In [3]:
from ngboost import NGBRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import joblib
import base64
import io
import json

# Define the to_dict method for the NGBoost model
def ngb_to_dict(model) -> dict:
    byte_stream = io.BytesIO()
    joblib.dump(model, byte_stream)
    byte_stream.seek(0)
    model_base64 = base64.b64encode(byte_stream.read()).decode('utf-8')
    out_dict = {
        "ngboost_model": model_base64
    }
    return out_dict

# Define the from_dict method for the NGBoost model
# @classmethod
def ngb_from_dict(cls, in_dict: dict) -> 'NGBRegressor':
    model_base64 = in_dict.get("ngboost_model")
    if model_base64 is None:
        raise ValueError("Model file must contain ngboost model at key: 'ngboost_model'")
    byte_stream = io.BytesIO(base64.b64decode(model_base64))
    ngboost_model = joblib.load(byte_stream)
    return ngboost_model

# Fetch the California Housing dataset
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Train the NGBoost model
ngb = NGBRegressor()
ngb.fit(X_train, y_train)

# Convert the model to a dictionary
model_dict = ngb_to_dict(ngb)

# Save the dictionary to a file
# with open('ngboost_model.json', 'w') as f:
#     json.dump(model_dict, f)

# # Load the dictionary from a file
# with open('ngboost_model.json', 'r') as f:
#     loaded_model_dict = json.load(f)

# Reconstruct the model from the dictionary
loaded_ngb = ngb_from_dict(NGBRegressor, model_dict)

# Use the loaded model to make predictions
predictions = loaded_ngb.predict(X_test)
print(predictions)


[iter 0] loss=1.5641 val_loss=0.0000 scale=1.0000 norm=1.1069
[iter 100] loss=1.1394 val_loss=0.0000 scale=1.0000 norm=0.7776
[iter 200] loss=0.9107 val_loss=0.0000 scale=1.0000 norm=0.6995
[iter 300] loss=0.7671 val_loss=0.0000 scale=1.0000 norm=0.6786
[iter 400] loss=0.6921 val_loss=0.0000 scale=1.0000 norm=0.6780
[0.66757509 1.07930075 3.86083596 ... 4.59621213 1.00234074 1.93741837]


In [7]:
# model_dict.values()

dict_values(['...'])